<a href="https://colab.research.google.com/github/yiruchen1993/nvidia_gtc_dli_rapids_2020/blob/section_notebooks%2Fproject/3_04_identify_risk_factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第三周: 確認感染的風險因子

<span style="color:red">
**更新**

再次感謝您的先前分析。接下來，我們將發佈公共衛生公告，警告個人應注意的特定感染風險因素。請告知哪些人群特徵與較高的感染率有關。
</span>

您的目標是通過比較感染人群和未感染人群來確定感染的潛在潛在人口地理和經濟風險因素。

## 載入相關套件

In [ ]:
import cudf
import cuml

## 載入資料

首先，將您在疫情爆發的第3週收到的數據載入到cuDF Dataframe 中。 數據位於`./data/week3.csv`。 對於此練習，您將需要數據的所有欄位資料。


In [ ]:
gdf = cudf.read_csv('./data/week3.csv')

## 按就業編碼計算感染率

將`infected`欄位轉換為`float32`型態。 對於未受感染的人，float32的`infected`值應為`0.0`，對於受感染的人，其值應為`1.0`。

In [ ]:
gdf.infected = gdf.infected.astype('float32')
gdf.head()

,age,sex,employment,infected
0,0,m,U,0.0
1,0,m,U,0.0
2,0,m,U,0.0
3,0,m,U,0.0
4,0,m,U,0.0


現在，列出一份就業類型及其相關的感染率清單，從最高感染率到最低感染率進行排序。

**注意**：每種就業類型的感染**率**應為該就業類型中被感染的總人數的百分比。因此，如果僱傭類型"X"有1000人，並且其中10人受到感染，則感染率.01。 如果僱傭類型"Z"有10,000人，並且其中有50人受到感染，則感染率將為.005，並且比"X"類型**低**，即使該僱傭類型中有更多的人被感染 。

In [ ]:
total_gdf = cudf.DataFrame()
total = gdf.employment.value_counts().to_frame()
total_gdf['total'] = total.employment
total_gdf['employ'] = total.index
total_gdf.head()

,total,employ
U,12459115,U
V,10098466,V
Z,7161907,Z
Q,3802602,Q
G,3549465,G


In [ ]:
infect_gdf = cudf.DataFrame()
infect = gdf.loc[gdf.infected==1,:].employment.value_counts().to_frame()
infect_gdf['infected_num'] = infect.employment
infect_gdf['employ'] = infect.index
infect_gdf.head()

,infected_num,employ
V,76648,V
Q,48505,Q
Z,40498,Z
P,18609,P
G,17561,G


In [ ]:
infect_rate_list = total_gdf.merge(infect_gdf)
infect_rate_list['infected_rate'] = infect_rate_list['infected_num']/infect_rate_list['total']
infect_rate_list = infect_rate_list.sort_values(by=['infected_rate'], ascending=False)
infect_rate_list.head()

,total,employ,infected_num,infected_rate
3,3802602,Q,48505,0.012756
11,1556575,I,16116,0.010354
1,10098466,V,76648,0.007590
5,3006149,P,18609,0.006190
2,7161907,Z,40498,0.005655


最後，請參考 `./data/code_guide.csv`中的就業守則指南，以解釋哪些就業類型感染率最高。

In [ ]:
code_gdf = cudf.read_csv('./data/code_guide.csv')

In [ ]:
code_gdf.loc[code_gdf.Code=='Q',:]

,Code,Field
14,Q,Human health & social work activities


## 通過就業代碼和性別計算感染率

我們想看看`sex`對感染率是否有影響，除了`employment`之外還是混雜起來。 同時按`employment`和`sex`進行分組，以得出這些類別交叉點的感染率。

## 進行評估

完成上述工作後，請訪問用於啟動此Jupyter Lab的*Launch Section*網頁。 向下滾動到啟動Jupyter Lab的位置，然後回答問題*第3週評估*。您可以通過訪問同一*Launch Section*頁面並單擊*Progress*頁面的鏈接來查看評估的總體進度。在*Progress*頁面上，如果您已成功回答所有評估問題，則可以單擊*Generate Certificate*以在課程中獲得證書。

![launch_task_page](./images/launch_task_page.png)

<div align="center"><h2>Optional: 重啟核心</h2></div>

如果您打算繼續在其他notebook工作，請關閉此notebook核心。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)